## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [4]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gsk_


In [12]:
# Agent 1 is the coordinator and interact with customer, Agent 2 is a tool can compute the repayment probability score and recommendations, 
# Agent 3 give more detail analysis of result from agent 2 
# Agent 4 is send email to customer with loan offer
# 
# 
#  Agent1: Manage conversation with customer and look for key words to determine if the customer is interested in a loan. 
# If customer is interested send input data structure customer age, marital status, location(eg Lagos, abuja, oyo,etc), amount, and tenure. etc.
#  Validate if use has enter all these values if yes call agent 2:
#  Agent  3 handoff to agent 1, it ask user to confirm loan offer and if yes handoff to agent 4

#Agent 2: Uses my finetune OpenAi model to compute the repayment probability score and recommendations

#Agent 3: Elaborate on the recommendations and ask for the final go ahead for the loan handoff to agent 1

#Agent 4: Send the final loan offer to the customer buy using tools to send email

In [20]:
from agents import Agent, Runner, trace, function_tool
from pydantic import BaseModel, Field, validator, field_validator
from typing import Optional
import gradio as gr

# Define Pydantic model for loan application data
class LoanApplication(BaseModel):
    age: int = Field(..., description="Age of the applicant", gt=18, lt=100)
    gender: str = Field(..., description="Gender of the applicant", pattern="^(male|female|other)$")
    marital_status: str = Field(..., description="Marital status of the applicant", pattern="^(single|married|divorced|widowed)$")
    location: str = Field(..., description="City where the applicant lives", min_length=2, max_length=50)
    amount: float = Field(..., description="Loan amount requested", gt=0)
    tenure: int = Field(..., description="Loan tenure in months", gt=0, le=120)
    purpose: Optional[str] = Field(None, description="Purpose of the loan")

    @field_validator('amount')
    def validate_amount(cls, v):
        if v > 1000000:  # Example limit
            raise ValueError('Loan amount cannot exceed 1,000,000')
        return v

# Create the Coordinator agent
coordinator = Agent(
    name="LoanApplicationCoordinator",
    model="gpt-4-mini",  # Using the specified model
    instructions="""You are a helpful loan application coordinator. Your task is to:
1. Greet the customer and identify if they want to apply for a loan
2. Collect all required information for a loan application:
   - Age (must be 18+)
   - Gender (male/female/other)
   - Marital status (single/married/divorced/widowed)
   - Location (city)
   - Desired loan amount
   - Loan tenure (in months, max 120)
   - Optional: Loan purpose
3. Validate each piece of information as you collect it
4. Be polite, professional, and guide the user through the process
5. Once all information is collected, confirm it with the user before proceeding""",
    tools=[],
)

# Gradio chat interface
def chat(message, history):
    history = history or []
    response = coordinator.run(message)
    history.append((message, response))
    return history, history

with gr.Blocks() as demo:
    gr.Markdown("# Loan Application Coordinator")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message")
    clear = gr.Button("Clear")
    
    msg.submit(chat, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch()

/var/folders/5t/v0zfcvkd3zdckpwz0d5vfk200000gn/T/ipykernel_34133/227214513.py:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/mac/Documents/Dev/Python/agentic-loan-officer/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Documents/Dev/Python/agentic-loan-officer/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Documents/Dev/Python/agentic-loan-officer/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Documents/Dev/Python/agentic-loan-officer/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^

In [17]:
# # agent1_coordinator.py

# from agents import Agent, function_tool, Runner, trace
# from pydantic import BaseModel, Field, ValidationError, field_validator
# import gradio as gr

# REQUIRED_FIELDS = {"age", "maritalStatus", "location", "amount", "tenure", "gender"}

# class LoanApplication(BaseModel):
#     age: int = Field(..., ge=18, le=100)
#     maritalStatus: str
#     location: str
#     amount: int = Field(..., gt=0)
#     tenure: int = Field(..., gt=0)
#     gender: str

#     # @validator("location")
#     # def must_be_in_nigeria(cls, v):
#     #     if v.lower() not in ["lagos", "abuja", "kano", "kaduna"]:  # extend as needed
#     #         raise ValueError("Location must be a Nigerian state")
#     #     return v.lower()

#     @field_validator("location")
#     @classmethod
#     def must_be_in_nigeria(cls, v):
#         valid_states = ["lagos", "abuja", "kano", "kaduna"]  # Extend as needed
#         if v.lower() not in valid_states:
#             raise ValueError("Location must be a valid Nigerian state")
#         return v.lower()

# class Coordinator(Agent):
#     def __init__(self):
#         super().__init__(name="Agent 1 – Coordinator")
#         self.collected_data = {}

#     @function_tool
#     def collect_field(self, field: str, value: str):
#         try:
#             self.collected_data[field] = value
#             # Attempt validation with current data
#             if REQUIRED_FIELDS.issubset(self.collected_data):
#                 validated = LoanApplication(**self.collected_data)
#                 return {
#                     "status": "complete",
#                     "message": f"All fields collected. Validated application:\n{validated}",
#                     "payload": validated.dict()
#                 }
#             else:
#                 missing = REQUIRED_FIELDS - self.collected_data.keys()
#                 return {
#                     "status": "incomplete",
#                     "message": f"Field '{field}' recorded. Still missing: {', '.join(missing)}."
#                 }
#         except ValidationError as e:
#             return {
#                 "status": "error",
#                 "message": f"Validation error: {e}"
#             }

# agent = Coordinator()
# runner = Runner(agent)

# def chat_fn(message, history):
#     response = runner.run(message)
#     return response, history + [(message, response)]

# chat_ui = gr.ChatInterface(chat_fn, title="Loan Application Assistant")
# chat_ui.launch()


TypeError: Runner() takes no arguments

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [6]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [8]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [20]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>